## Building a cell & network from scratch using NeuroML and NeuroMLlite

In [8]:
import neuroml
import neuroml.writers as writers
from pyneuroml import pynml

cell_id = 'BallAndStick'

nml_cell_doc = neuroml.NeuroMLDocument(id=cell_id)

cell = neuroml.Cell(id=cell_id)
nml_cell_doc.cells.append(cell)

cell.morphology = neuroml.Morphology(id='morphology')  


def add_segment(cell, prox, dist, name=None, parent=None, group=None):
    
    p = neuroml.Point3DWithDiam(x=prox[0], y=prox[1], z=prox[2], diameter=prox[3])
    d = neuroml.Point3DWithDiam(x=dist[0], y=dist[1], z=dist[2], diameter=dist[3])

    segid = len(cell.morphology.segments)
    
    sp = neuroml.SegmentParent(segments=parent.id) if parent else None
    segment = neuroml.Segment(id=segid, proximal = p,  distal = d, parent = sp)
    
    if name:
        segment.name=name
        
    if group:
        seg_group = None
        for sg in cell.morphology.segment_groups:
            if sg.id==group: seg_group = sg
                
        if seg_group is None:
            neuro_lex_id = None
            if group=="axon_group": neuro_lex_id="GO:0030424" # See http://amigo.geneontology.org/amigo/term/GO:0030424
            if group=="soma_group": neuro_lex_id="GO:0043025"
            if group=="dendrite_group": neuro_lex_id="GO:0030425"
            
            seg_group = neuroml.SegmentGroup(id=group,neuro_lex_id=neuro_lex_id)
            
            cell.morphology.segment_groups.append(seg_group)
            
        seg_group.members.append(neuroml.Member(segments=segment.id))
    
    cell.morphology.segments.append(segment)
    
    return segment

soma =  add_segment(cell,(0,0,0,20),(0,0,0,20),name='soma',group='soma_group')
dend1 = add_segment(cell,(0,0,0,5),(30,0,0,5),name='dend1',group='dendrite_group',parent=soma)
dend2 = add_segment(cell,(30,0,0,4),(60,0,0,4),name='dend2',group='dendrite_group',parent=dend1)


pynml.summary(nml_cell_doc, verbose=True)

nml_file = cell.id+'.cell.nml'

writers.NeuroMLWriter.write(nml_cell_doc,nml_file)

print("Saved cell file to: "+nml_file)

*******************************************************
* NeuroMLDocument: BallAndStick
*
*
*  Cell: BallAndStick
*    <Segment|0|soma>
*      Parent segment: None (root segment)
*      (0.0, 0.0, 0.0), diam 20.0um -> (0.0, 0.0, 0.0), diam 20.0um; seg length: 0.0 um
*      Surface area: 1256.6370614359173 um2, volume: 4188.790204786391 um3
*    <Segment|1|dend1>
*      Parent segment: 0
*      (0.0, 0.0, 0.0), diam 5.0um -> (30.0, 0.0, 0.0), diam 5.0um; seg length: 30.0 um
*      Surface area: 471.23889803846896 um2, volume: 589.0486225480861 um3
*    <Segment|2|dend2>
*      Parent segment: 1
*      (30.0, 0.0, 0.0), diam 4.0um -> (60.0, 0.0, 0.0), diam 4.0um; seg length: 30.0 um
*      Surface area: 376.99111843077515 um2, volume: 376.99111843077515 um3
*    Total length of 3 segments: 60.0 um; total area: 2104.8670779051613 um2
*  
*    SegmentGroup: soma_group,	1 member(s),	0 included group(s);	contains 1 segment, id: 0
*    SegmentGroup: dendrite_group,	2 member(s),	0 included gro